In [2]:
## MNIST with Keras
## 텐서플로우를 부품으로 사용하는 keras는 보다 높은 레벨로 기술할 수 있다.

from keras.datasets import mnist

## 손으로 쓴 숫자의 농담 화상 데이터이다.
## 28x28 픽셀의 60000개의 훈련 데이터와 10000개의 테스트 데이터로 구성되어 있다.
(X_train, y_train), (X_test, y_test) = mnist.load_data()

## 데이터 가공
## 우선, 입력을 일반적인 화상인식 CNN의 표준으로 변환한다
## 통상의 화상인식은 컬러 화상을 대상으로 하여 1장의 화상은 세로화소수 x 가로화소수 x 색의 수: 총 3차원 텐서로 구성된다.
## 입력 데이터 전체는 제일 앞에 화상의 매수를 추가한 4차원 텐서로 구성된다.

## 주어진 데이터는 흑백이미지로, 화상의 매수 x 세로화소수 x 가로화소수로 나타나서 색의 수 1을 하나의 차원로 더하여 4차원 텐서로 변환한다.

img_rows, img_cols = 28, 28

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1) ## 60000 x 28 x 28 x 1
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1) ## 10000 x 28 x 28 x 1
input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')/255     ## 32비트 부동소수점으로 변환하고 255로 나누어 0~1 사이의 값으로 정규화한다.
X_test = X_test.astype('float32')/255

## 레이블을 원-핫 인코딩으로 변환한다.
## 원-핫 인코딩이란, 0~9까지의 정수형 레이블을 10개의 요소를 가진 벡터로 변환하는 것이다.
## 예를 들어, 0은 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]으로 변환된다.
## 이는 0부터 9까지의 정수형 레이블을 가진 데이터를 10개의 클래스를 가진 분류 문제로 변환하는 것이다.

from keras.utils import to_categorical

Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

## 모델 구축
## keras는 Sequential 모델을 사용하여 모델을 구축한다.
## Sequential 모델은 레이어를 선형으로 쌓아 구축하는 모델이다.

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

n_out = len(Y_train[0]) ## 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape)) ## 32개의 필터를 사용하여 3x3의 커널을 적용한다.
model.add(MaxPooling2D(pool_size=(2, 2))) ## 2x2의 맥스 풀링을 적용한다.
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu')) ## 64개의 필터를 사용하여 3x3의 커널을 적용한다.
model.add(MaxPooling2D(pool_size=(2, 2))) ## 2x2의 맥스 풀링을 적용한다.
model.add(Flatten()) ## 1차원으로 변환한다.
model.add(Dense(128, activation='relu')) ## 128개의 노드를 가진 완전연결층을 적용한다.
model.add(Dense(n_out, activation='softmax')) ## 10개의 노드를 가진 완전연결층을 적용한다.
model.summary()


2023-07-10 16:15:32.849619: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-10 16:15:32.852709: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 16:15:32.903208: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-10 16:15:32.905920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-10 16:15:33.956561: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

11490434/11490434 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 13, 13, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 5, 5, 64)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                         

In [3]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.fit(X_train, Y_train, epochs=5, batch_size=200)
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

Epoch 1/5
300/300 [==============================] - 10s 31ms/step - loss: 0.2490 - accuracy: 0.9244
Epoch 2/5
300/300 [==============================] - 9s 29ms/step - loss: 0.0613 - accuracy: 0.9813
Epoch 3/5
300/300 [==============================] - 9s 28ms/step - loss: 0.0405 - accuracy: 0.9872
Epoch 4/5
300/300 [==============================] - 9s 29ms/step - loss: 0.0303 - accuracy: 0.9907
Epoch 5/5
300/300 [==============================] - 9s 29ms/step - loss: 0.0230 - accuracy: 0.9926
Test loss:  0.036492105573415756
Test accuracy:  0.9882000088691711
